모듈 및 라이브러리 불러오기

In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 26.3 MB/s eta 0:00:00


In [2]:
!pip install transformers pytorch_lightning tokenizers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 14.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [3]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device('cpu')

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
import random
import os
import re
import torch
import tensorflow as tf

from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import ElectraForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from keras.preprocessing.sequence import pad_sequences
from transformers import AutoModelForSequenceClassification, pipeline

import torch.nn as nn # 분류 헤드 수정 - KoElectra 모델의 출력층을 6개의 클래스로 분류할 수 있도록 수정

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


모델 불러오기

In [ ]:
'''# parameter
max_len = 64
batch_size = 32'''

In [6]:
from transformers import ElectraTokenizer
model_name = "monologg/koelectra-base-v3-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=6)  # 감정 종류가 6개라고 가정

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
#model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)
#model.classifier.out_proj = nn.Linear(in_features=768, out_features=6)

PATH ='/content/drive/MyDrive/0000/Class/졸프/model_fin/model_KoElectra_0527.pt'
model.load_state_dict(torch.load(PATH, map_location=device))
model.eval()

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

모델 예측

In [8]:
# 입력 문장 예측 함수
def predict(predict_sentence):
    model.eval()

    # 텍스트 클랜징 함수 : 특수문제 제거 및 여러개의 공백 하나로 줄이기
    predict_sentence = re.sub(r'[^가-힣A-Za-z0-9 ]', '', predict_sentence)
    predict_sentence = re.sub(r'\s+', ' ', predict_sentence).strip()
    print(predict_sentence)
    # 입력 문장 토큰화
    inputs = tokenizer(
        predict_sentence,
        return_tensors="pt",
        max_length=64,  # 최대 길이 설정
        padding="max_length",
        truncation=True
    )

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # 모델 예측
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 소프트맥스 함수로 확률 계산
    probs = F.softmax(logits, dim=1)

    # 결과 해석
    probs = probs.detach().cpu().numpy()[0]
    pred_label = np.argmax(probs)

    emotion_labels = ["기쁨", "당황", "분노", "불안", "상처", "슬픔"]
    pred_emotion = emotion_labels[pred_label]

    # 각 감정에 대한 확률을 퍼센트로 변환하여 출력
    for i, (label, prob) in enumerate(zip(emotion_labels, probs)):
        print(f"{label}: {prob * 100:.2f}%")

    # 가장 높은 확률을 차지하는 감정 출력
    print(f"\n>> 입력하신 내용에서 가장 느껴지는 감정은 '{pred_emotion}'이며, 확률은 {probs[pred_label] * 100:.2f}%입니다.")

In [9]:
sentence = '모델이 안돌아간다. 매일같이 날 화나게 한다. 정말이지 열라 짱난다. \
                    누군가가 제발 간절히 도와줬으면 좋겠다. 오늘도 힘들었다.'
predict(sentence)

모델이 안돌아간다 매일같이 날 화나게 한다 정말이지 열라 짱난다 누군가가 제발 간절히 도와줬으면 좋겠다 오늘도 힘들었다
기쁨: 0.06%
당황: 0.06%
분노: 99.47%
불안: 0.13%
상처: 0.07%
슬픔: 0.21%

>> 입력하신 내용에서 가장 느껴지는 감정은 '분노'이며, 확률은 99.47%입니다.


In [10]:
sentence = '나 진짜 객체 때문에 화병날 것 같아. 너무 화가 나. 화가 날때는 무엇을 해야 할까? \
                    남친이랑 통화해야겠다. 통화 좀 하고 올게.'
predict(sentence)

나 진짜 객체 때문에 화병날 것 같아 너무 화가 나 화가 날때는 무엇을 해야 할까 남친이랑 통화해야겠다 통화 좀 하고 올게
기쁨: 4.22%
당황: 2.09%
분노: 58.40%
불안: 0.86%
상처: 0.64%
슬픔: 33.79%

>> 입력하신 내용에서 가장 느껴지는 감정은 '분노'이며, 확률은 58.40%입니다.


In [11]:
diary = pd.read_csv("/content/drive/MyDrive/0000/Class/졸프/Dataset/EveryonesCorpus_diary.csv")
diary = diary[['text']]
diary

,text
0,일이 하기 싫다. 오늘은 늦게 일어나서 지각을 할 뻔했지만 지각을 하지 않았다. 다...
1,어른들의 말이 문득 떠오른다... 크면 다 알게될꺼라는 말들 이제야 문득문득 어릴때...
2,요즘 코로나 바이러스가 사회를 공포로 휩싸이게 하고 있다. 처음에는 대수롭지 않게 ...
3,9일 하루 함께 있었는데 내게 힘이 생겼다. 다시 누군가를 그대로 용납할 수 있는 ...
4,'혼자서? 여행을간다고?' 여행을 혼자간다고하면 제일 첫번째로 듣는 질문이다. 그 ...
...,...
292,아침에 너무 기분 좋고 상쾌하게 눈 떴는데 9시 좀 넘은 시간이었다. 이 기세 그대...
293,"당신께 거룩한 입맞춤을 합니다. 나는 이미 내 마음을 당신께 드렸고, 그것만으로 충..."
294,누군가는 내가 사는 이곳에 여행을 온다. 매일 가는 두부집이 맛집이라고 주말만 되면...
295,요즘 날이 추워 길냥이들의 물이 얼어 냥이들이 마시지 못하고 있다. 오늘도 걱정이 ...


In [12]:
sentence = diary.text[2]
predict(sentence)

요즘 코로나 바이러스가 사회를 공포로 휩싸이게 하고 있다 처음에는 대수롭지 않게 생각하던 사람들도 이제는 심각성을 깨닫고 사람들의 대화주제도 늘 코로나 바이러스이다 코로나 바이러스는 우리의 일상을 순식간에 바꿔버렸다 4살 아들과 함께 매주 주말이면 어디든 나가는 우리집은 주말 내내 집콕을 하며 시간을 보내도록 바꾸었고 거리의 사람들과 관공서 음식점의 음식하시는 분들까지 마스크로 채워버렸다 그리고 타인에 대한 경계 불안감을 만들었다 요즘 4살 아들에게 이런 상황을 설명하며 항상 마스크를 껴야 한다고 설득하고 있다 평소 내가 불만이었던 집돌이 남편과 집돌이 아들의 성향이 고마워졌다 사람들의 불안감은 마스크에서 잘 나타난다 마스크는 하루밤 자고 나면 가격이 올라있다 어제의 가격이 가장 저렴한 가격이 되어 있다 이런 상황속에서 마스크를 대량구매해 둔 것을 되파는 사람들도 생겨나고 있다 대단한 생존력이 아닐수 없다 항상 위기에서 기회가 온다고 하는데 기회를 잘 포착하는 능력자들에 틀림 없다 더 이상 확진자가 늘어나지 않고 사망자 없이 잘 종식되고 마무리되길 바란다
기쁨: 0.50%
당황: 0.13%
분노: 4.72%
불안: 91.69%
상처: 0.68%
슬픔: 2.28%

>> 입력하신 내용에서 가장 느껴지는 감정은 '불안'이며, 확률은 91.69%입니다.


In [13]:
sentence = diary.text[76]
predict(sentence)

기상시간은 6시 지난 밤에는 내내 차가운 물만 쏟아지더니 간밤에 돌렸다는 발전기가 열심히 일을 한 모양인지 따뜻한 물이 나오고 있었다 하루치 샤워는 포기하려고 했지만 따뜻한 물이 나온다면 감사한 일이지 소문내지 않고 호다닥 샤워를 하고 나왔다 대한민국 전기수도정수시설 감사합니다 가이드가 아침식사로 빵과 달걀부침 편으로 썬 오이랑 토마토 엷게 내린 커피를 준비했다 하루 일정이 바쁘지 아침부터 서두르게 하더라니 정작 오늘치 장을 보기 위해 들른 가게가 아직 문을 열지 않았다 주차장에 차를 대어두고 이래저래 스트레칭도 하고 즉석에서 예의 나는 잘 모르는 인싸춤이라고 하는 걸 몇 가지 배웠다 인싸들 세상 참 어렵게 산다 결국은 할 줄 아는 국민체조로 귀결 와이파이를 잡아서 국민체조 완곡 파일을 다운받아서 재생하였다 동그랗게 둘러서서 국민체조 동작을 하는데 나만 좌우가 반대다 십년을 넘게 반복해도 익혀지지가 않는 모양이다 오늘 이동은 487킬로미터 차로 여덞시간을 달려야 한다 그래도 뭐 내가 운전하는 거 아니니까 라고 가볍게 마음먹고 출발하였다 물론 얼마 지나지 않아 그 생각이 뭘 모르는 나님의 짧은 생각이었다는 것을 알게된다 내가 오늘날 누리고 있는 인프라가 얼마나 중요하고 기본적인 것인지를 알게 되었는데 그것은 바로 포장도로 비포장도로에서의 운전은 운전자나 동승자 모두가 어마어마하게 고생이라는 걸 새삼 느꼈다 나 어릴적에만 해도 없던 국도가 뚫리고 오토바이로 논길 다니고 하던 기억이 분명 있었는데 오랫동안 잊고 있었다 그래 우리나라는 땅이 꽤 좁고 인구밀도가 높은 편이라 연결망 하나는 어마어마 발전했지 않은가 매번 불평하는 대로 땅덩어리는 조그만 데에 몰려사느라 땅값은 비싸고 자원도 안나고 저기 어느 나라는 위치를 잘 타고 난 덕으로 힘들게 노동하지 않아도 천연자원으로 뽕을 뽑고 산다는데 이런 이야기를 40자 트위터에 싸지르곤 했었는데 말이다 단군할아버지가 부동산은 모르셨군요 이런 거 이곳처럼 위도는 높아서 춥고 땅은 넓고 인구밀도는 낮고 소양말낙타는 행복하고 사람

In [14]:
sentence = "새해가 되면서 목표한 것 중에 하나였던 매일 그림책 읽어주기. 사실 매일 읽어주면 나도 아이들도 지겨울까봐 살짝 꾀를 내어 월, 수, 금요일에만 읽어주려고 했는데 다행히 아이들이 좋아해서 매일 읽어주게 되었다. 자기 전에 아이들이 직접 고른 그림책을 들고 와서 읽고나면 잠자리에 드는, 참으로 이상적인 그림이었는데. 슬슬 문제가 생기기 시작했다. 바로 슬픈, 감동적인 그림책을 읽다가 엉엉 울어버린다는 것. \
              그것도 아이들이 아닌 읽어주는 어른인 내가 말이다. 처음엔 창피해서 꾹 참아보려 했는데 이게 어느 정점을 딱 지나면 나도 모르게 주르륵 흘러내려서 들키지 않을 수가 없게 된다는 문제였다. 이런 상황을 몇 번 겪다 보니 뭔가 슬프고 감동적인 순간이 오면 이 녀석들이 책은 안 보고 내 얼굴을 살피게 되고. 뭔가 대단한 것이라도 찾아낸 것처럼 '언니~ 엄마 울어!'라고 소리치는 둘째, '어이구~ 감수성 풍부한 우리 엄마~'라고 다독여주는 첫째까지. 그 가운데에서 부끄러운 건 나 혼자만의 몫이었다. 내가 왜 이럴까? 나이가 들어서 그런가? 스스로에게 물어봤지만, 아니다. 돌이켜보니 아쉽게도 난 원래 그랬었다. 작년에도 우연히 빌려온 '고녀석 맛있겠다'를 읽고 펑펑 울며 그 작가의 다른 책까지 모조리 읽고 눈이 붓도록 펑펑 울었던 기억. 그래도 그때는 아이들을 학교에 다 보내놓은 상태였는데. 이제는 아이들 앞에서 울고 있으니..한심하기도 하면서 부끄럽기도 하고. 눈물을 참지 못하는 내가 슬프기도 했다. 하지만 눈물이 나면 우는 것도(그게 설령 아이들 앞이라고 해도) 어떻게 보면 감정에 솔직한 것이니까..괜찮은거라고 위안을 삼아보지만..그래도 부끄러운 것은 사라지지 않는다. 그래도 아이들이 울고 있는 나를 토닥여주니 어떻게보면 또 의외의 성과가 있는 것도 같고.(사실 아무리 예쁘게 포장을 해봐도 창피하고 부끄럽다.) 아직 한참 남은 올 한해. 아이들과 함께 열심히 읽고, 열심히 느껴야지. 부디 12월쯤 되면 슬프고 감동적인 내용을 읽어도 눈물이 안 날 정도로 무뎌져 있기를. 아니면 우는게 나뿐아니라 아이들도 같이 울 수 있도록 아이들의 감수성도 쑥쑥 자라나기를 살짝쿵 바래본다."
predict(sentence)

새해가 되면서 목표한 것 중에 하나였던 매일 그림책 읽어주기 사실 매일 읽어주면 나도 아이들도 지겨울까봐 살짝 꾀를 내어 월 수 금요일에만 읽어주려고 했는데 다행히 아이들이 좋아해서 매일 읽어주게 되었다 자기 전에 아이들이 직접 고른 그림책을 들고 와서 읽고나면 잠자리에 드는 참으로 이상적인 그림이었는데 슬슬 문제가 생기기 시작했다 바로 슬픈 감동적인 그림책을 읽다가 엉엉 울어버린다는 것 그것도 아이들이 아닌 읽어주는 어른인 내가 말이다 처음엔 창피해서 꾹 참아보려 했는데 이게 어느 정점을 딱 지나면 나도 모르게 주르륵 흘러내려서 들키지 않을 수가 없게 된다는 문제였다 이런 상황을 몇 번 겪다 보니 뭔가 슬프고 감동적인 순간이 오면 이 녀석들이 책은 안 보고 내 얼굴을 살피게 되고 뭔가 대단한 것이라도 찾아낸 것처럼 언니 엄마 울어라고 소리치는 둘째 어이구 감수성 풍부한 우리 엄마라고 다독여주는 첫째까지 그 가운데에서 부끄러운 건 나 혼자만의 몫이었다 내가 왜 이럴까 나이가 들어서 그런가 스스로에게 물어봤지만 아니다 돌이켜보니 아쉽게도 난 원래 그랬었다 작년에도 우연히 빌려온 고녀석 맛있겠다를 읽고 펑펑 울며 그 작가의 다른 책까지 모조리 읽고 눈이 붓도록 펑펑 울었던 기억 그래도 그때는 아이들을 학교에 다 보내놓은 상태였는데 이제는 아이들 앞에서 울고 있으니한심하기도 하면서 부끄럽기도 하고 눈물을 참지 못하는 내가 슬프기도 했다 하지만 눈물이 나면 우는 것도그게 설령 아이들 앞이라고 해도 어떻게 보면 감정에 솔직한 것이니까괜찮은거라고 위안을 삼아보지만그래도 부끄러운 것은 사라지지 않는다 그래도 아이들이 울고 있는 나를 토닥여주니 어떻게보면 또 의외의 성과가 있는 것도 같고사실 아무리 예쁘게 포장을 해봐도 창피하고 부끄럽다 아직 한참 남은 올 한해 아이들과 함께 열심히 읽고 열심히 느껴야지 부디 12월쯤 되면 슬프고 감동적인 내용을 읽어도 눈물이 안 날 정도로 무뎌져 있기를 아니면 우는게 나뿐아니라 아이들도 같이 울 수 있도록 아이들의 감수성도 쑥쑥 자라나기를